# Setup

In [1]:
import os
home_dir=os.path.expanduser('~')
os.chdir(f"{home_dir}/nexus_correlation_discovery/")
%pip install -e . # install nexus

Obtaining file:///Users/hrisheeshastri/nexus_correlation_discovery
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for nexus (pyproject.toml) ... done
  Created wheel for nexus: filename=nexus-0.0.1-0.editable-py3-none-any.whl size=4715 sha256=6f2accb892ebbb5c9ed231907b466e38886ae45b2e6c6744e0177db2b448fa1e
  Stored in directory: /private/var/folders/hp/t2r71nqs2xvdrwnt5vj3hbmw0000gn/T/pip-ephem-wheel-cache-j3pkyomp/wheels/d3/c0/9c/640893bb9899cb23b83f1455fa5b1f206c6bdd99cc6eb68942
Successfully built nexus
  Attempting uninstall: nexus
    Found existing installation: nexus 0.0.1
    Uninstalling nexus-0.0.1:
      Successfully uninstalled nexus-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
nexus_root_dir="/home/cc/nexus_correlation_discovery/"
from nexus.utils.time_point import TEMPORAL_GRANU
from nexus.utils.coordinate import SPATIAL_GRANU
from demo.demo_ui import show_df
from nexus.nexus_api import API
from nexus.utils.data_model import Variable
from sklearn import linear_model
import warnings
from nexus.corr_analysis.graph.graph_utils import filter_on_signals
from nexus.utils.io_utils import load_corrs_from_dir
from demo.cluster_utils import CorrCommunity
from demo.demo_ui import show_communities
import pickle

warnings. filterwarnings('ignore')

use_qgrid = False

# Create Nexus API

In [3]:
conn_str = f'data/quickstart.db'
nexus_api = API(conn_str, data_sources=['chicago_zipcode', 'asthma', 'chicago_factors'])

## Browse Data Assets

You can use Nexus to browse data catalog and an individual dataset.

### Download output data products

Every dataframe displayed by Nexus is associated with a download button. After the button is clicked, the dataframe will be downloaded to the root directory of `nexus_correlation_discovery`. The name of the downloaded dataframe is the one you specified in the name field of `show_df`.

In [4]:
# show catalog
catalog = nexus_api.show_catalog()

show_df(catalog, name="catalog", use_qgrid=use_qgrid)

,id,name,link
0,ijzp-q8t2,Crimes - 2001 to Present,
1,ydr8-5enu,Building Permits,
2,s6ha-ppgi,Affordable Rental Housing Developments,
3,4ijn-s7e5,Food Inspections,
4,z8bn-74gv,Police Stations,
...,...,...,...
337,a5x7-5y4m,Contractor Business Diversity Program Submissions,
338,asthma,asthma,unknown
339,chicago_zipcode_population,chicago_zipcode_population,https://healthdata.gov/dataset/Chicago-Populat...
340,chicago_income_by_zipcode,chicago_income_by_zipcode,https://simplemaps.com/city/chicago/zips/incom...


Button(description='Download Data', style=ButtonStyle())

In [5]:
# show original dataset
dataset_id = '4u6w-irs9'
df, link=nexus_api.show_raw_dataset(id=dataset_id)
print(link)
show_df(df, name=dataset_id, use_qgrid=use_qgrid)

,STORE NAME,LICENSE ID,ACCOUNT NUMBER,SQUARE FEET,BUFFER SIZE,ADDRESS,ZIP CODE,COMMUNITY AREA NAME,COMMUNITY AREA,WARD,CENSUS TRACT,CENSUS BLOCK,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION
0,WINDY CITY INC,24289,9833,4225,B,324 S CICERO AVE,60644,AUSTIN,25,24,17031252202,1.703125e+14,1.144409e+06,1.897933e+06,41.875944,-87.745250,"(41.8759443554, -87.7452502275)"
1,ONE STOP FOOD & LIQUOR STORE,1094,205,58000,A,4301 S LAKE PARK AVE,60653,KENWOOD,39,4,17031390100,1.703139e+14,1.184508e+06,1.876712e+06,41.816865,-87.598689,"(41.8168651481, -87.5986888442)"
2,D & A FOOD & LIQUOR,46444,7945,5500,B,1601 W GARFIELD BLVD,60636,WEST ENGLEWOOD,67,15,17031670300,1.703167e+14,1.166597e+06,1.868071e+06,41.793554,-87.664638,"(41.793554363, -87.6646375512)"
3,ULTRA FOODS,1741082,308026,87600,A,3250 W 87TH ST,60652,ASHBURN,70,18,17031700402,1.703170e+14,1.156152e+06,1.846754e+06,41.735274,-87.703511,"(41.7352740895, -87.7035112835)"
4,PETE'S FRESH SUPERMARKET,1771242,312119,75000,A,2526 W CERMAK RD,60608,SOUTH LAWNDALE,30,28,17031840700,1.703184e+14,1.159779e+06,1.889354e+06,41.852101,-87.689051,"(41.8521006532, -87.6890511106)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,"CARNICERIA LA HACIENDA, INC.",1492826,276948,4000,B,3459 W 59TH ST,60629,CHICAGO LAWN,66,16,17031660400,1.703166e+14,1.154082e+06,1.865233e+06,41.786024,-87.710605,"(41.786024401, -87.7106047774)"
487,CICERO DELI INC.,20884,8639,3600,B,3206 N CICERO AVE,60641,PORTAGE PARK,15,30,17031151002,1.703115e+14,1.143769e+06,1.920924e+06,41.939047,-87.747022,"(41.93904667, -87.7470219655)"
488,K BUENA,1999075,65396,3500,B,4322 W MARQUETTE RD,60629,WEST LAWN,65,13,17031650301,1.703165e+14,1.148570e+06,1.859858e+06,41.771383,-87.730953,"(41.7713833198, -87.7309533689)"
489,JEWEL FOOD STORE #3210,1333,259,23845,A,1210 N CLARK ST,60610,NEAR NORTH SIDE,8,42,17031080202,1.703108e+14,1.175275e+06,1.908485e+06,41.904265,-87.631603,"(41.9042652283, -87.6316032946)"


Button(description='Download Data', style=ButtonStyle())

In [6]:
"""
Show aggregated dataset

For example, 4u6w-irs9_location_6 is an aggreagted table, 
which is created over original table 4u6w-irs9 by aggregating its spatial attribute `location` 
to the zipcode granularity (zipcode granularity is mapped to 6 in Nexus).
"""
agg_tbl_name = '4u6w-irs9_location_6'
df = nexus_api.show_agg_dataset(agg_tbl_name)
show_df(df, name=agg_tbl_name, use_qgrid=use_qgrid)

,val,avg_square_feet,count
0,60639.0,8889.320,25.0
1,60614.0,12457.583,12.0
2,60619.0,11713.273,11.0
3,60624.0,6833.125,8.0
4,60646.0,9958.750,4.0
5,60610.0,19874.333,6.0
6,60644.0,9532.429,7.0
7,60652.0,51950.000,2.0
8,60608.0,15612.688,16.0
9,60625.0,10670.429,14.0


Button(description='Download Data', style=ButtonStyle())

In [19]:
import duckdb
"""
use duckdb to browse datasets
"""
conn = duckdb.connect(conn_str, read_only=True)
df = conn.execute("SELECT * FROM 'asthma_Zip5_6'").df()
conn.close()

# Find correlations from an input table

## Inputs:
- `dataset`: dataset id
- `temporal_granularity`: temporal granularity
- `spatial_granularity`: spatial granularity
- `overlap_threshold`: overlap threshold for joinable detection
- `correlation_threshold`: correlation coefficient threshold
- `correlation_type`: correlation type: `pearson`, `spearman`, `kendall`
- `control_variables`: variables that you want to control for. When `control_variables` is specified, partial(conditional) correlations are calculated w.r.t control varaibles.

## Outputs:
A list of of correlations, and each correlation has the following attributes.

- `table_id`: table id, `table_name`: table name

- `agg_table`: the table name of the aggregated table. For example, 4u6w-irs9_location_6 is an aggreagted table, which is created over original table 4u6w-irs9 by aggregating its spatial attribute `location` to the zipcode granularity (zipcode granularity is mapped to 6 in Nexus).

- `agg_attr`: the attribute after aggregation.

- `correlation coefficient` is the correlation coefficient.

- `p value` is the p value for the correlation

- `original_attribute_missing_ratio` is the fraction of missing values in the original attribute before any aggregation.

- `number of samples` is the number of rows used to calculate the correlation.

- `spatio-temporal key type` indicates whether this correlation by spatial alignment or temporal alignment or both.


In this example, the input is the asthma dataset. We align the input with tables from Chicago open data and compute correlations. Tables from chicago open data originally have the spatial granularity of geo-coordinate. We aggregate them to the zipcode level and apply aggregate functions "avg" and "count". For example, if you see an attribute named "avg_basketball_courts", it means the original attribute is "basketball_courts" and function "avg" is applied. The attribute after aggregation is named "avg_basketball_courts". In the displayed dataframe, you can perform sorting on one dimension, filtering rows using keywords, and etc.

In [8]:
dataset = 'asthma'
# asthma data only has spatial attribute, thus the temporal granularity is set to ALL.
temporal_granularity, spatial_granularity = TEMPORAL_GRANU.ALL, SPATIAL_GRANU.ZIPCODE
overlap_threshold = 5
correlation_threshold = 0.5
# you can change correlation_type to 'spearman' or 'kendall'
correlations = nexus_api.find_correlations_from(dataset, temporal_granularity, spatial_granularity, 
                                      overlap_threshold, correlation_threshold, 
                                      correlation_type="pearson")
show_df(correlations, name='asthma_corrs', use_qgrid=use_qgrid)

total number of correlations: 234


,table_id1,table_name1,agg_table1,agg_attr1,original_attr1_missing_ratio,table_id2,table_name2,agg_table2,agg_attr2,original_attr2_missing_ratio,correlation coefficient,p value,number of samples,spatio-temporal key type
0,asthma,asthma,asthma_Zip5_6,avg_enc_asthma,0.0,ijzp-q8t2,Crimes - 2001 to Present,ijzp-q8t2_location_6,count,0.00,0.560,0.0,50,spatial
1,asthma,asthma,asthma_Zip5_6,avg_encAsthmaExac,0.0,ijzp-q8t2,Crimes - 2001 to Present,ijzp-q8t2_location_6,count,0.00,0.540,0.0,50,spatial
2,asthma,asthma,asthma_Zip5_6,avg_enc_asthma,0.0,85ca-t3if,Traffic Crashes - Crashes,85ca-t3if_location_6,avg_injuries_total,0.00,0.564,0.0,50,spatial
3,asthma,asthma,asthma_Zip5_6,avg_encAsthmaExac,0.0,85ca-t3if,Traffic Crashes - Crashes,85ca-t3if_location_6,avg_injuries_total,0.00,0.578,0.0,50,spatial
4,asthma,asthma,asthma_Zip5_6,avg_enc_asthma,0.0,aksk-kvfp,City-Owned Land Inventory,aksk-kvfp_location_6,avg_square_footage_city_estimate,0.82,0.679,0.0,50,spatial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,asthma,asthma,asthma_Zip5_6,avg_encAsthmaExac,0.0,n26f-ihde,Transportation Network Providers - Trips (2023-),n26f-ihde_pickup_centroid_location_6,avg_trips_pooled,0.00,0.547,0.0,49,spatial
230,asthma,asthma,asthma_Zip5_6,avg_enc_asthma,0.0,n26f-ihde,Transportation Network Providers - Trips (2023-),n26f-ihde_dropoff_centroid_location_6,avg_trips_pooled,0.00,0.541,0.0,49,spatial
231,asthma,asthma,asthma_Zip5_6,avg_encAsthmaExac,0.0,n26f-ihde,Transportation Network Providers - Trips (2023-),n26f-ihde_dropoff_centroid_location_6,avg_trips_pooled,0.00,0.535,0.0,49,spatial
232,asthma,asthma,asthma_Zip5_6,avg_enc_asthma,0.0,chicago_income_by_zipcode,chicago_income_by_zipcode,chicago_income_by_zipcode_zipcode_6,avg_income_household_median,0.00,-0.507,0.0,50,spatial


Button(description='Download Data', style=ButtonStyle())

## Control for variables

We got 234 correlations for the asthma dataset. After browsing several correlations, we realize that "poverty" might be driving these correlations after going through these correlations. Thus, we want to control for the income level of each zipcode when calculating correlations. To achieve that, users can specify variables that they want to control for in the `control_variables` field. 

In [9]:
dataset = 'asthma'
temporal_granularity, spatial_granularity = TEMPORAL_GRANU.ALL, SPATIAL_GRANU.ZIPCODE
overlap_threshold = 5
correlation_threshold = 0.5
control_variables = [Variable('chicago_income_by_zipcode_zipcode_6', 'avg_income_household_median')]
df_control = nexus_api.find_correlations_from(dataset, temporal_granularity, spatial_granularity, 
                                              overlap_threshold, correlation_threshold, 
                                              correlation_type="pearson", control_variables=control_variables)
show_df(df_control, name='asthma_corrs_control_income', use_qgrid=use_qgrid)

total number of correlations: 56


,table_id1,table_name1,agg_table1,agg_attr1,original_attr1_missing_ratio,table_id2,table_name2,agg_table2,agg_attr2,original_attr2_missing_ratio,correlation coefficient,p value,number of samples,spatio-temporal key type
0,asthma,asthma,asthma_Zip5_6,avg_enc_asthma,0.0,85ca-t3if,Traffic Crashes - Crashes,85ca-t3if_location_6,avg_injuries_fatal,0.00,-1.000,0.000,50,spatial
1,asthma,asthma,asthma_Zip5_6,avg_encAsthmaExac,0.0,85ca-t3if,Traffic Crashes - Crashes,85ca-t3if_location_6,avg_injuries_fatal,0.00,-1.000,0.000,50,spatial
2,asthma,asthma,asthma_Zip5_6,avg_AttackPer,0.0,85ca-t3if,Traffic Crashes - Crashes,85ca-t3if_location_6,avg_injuries_fatal,0.00,-1.000,0.000,50,spatial
3,asthma,asthma,asthma_Zip5_6,avg_enc_asthma,0.0,aksk-kvfp,City-Owned Land Inventory,aksk-kvfp_location_6,avg_square_footage_city_estimate,0.82,0.539,0.000,50,spatial
4,asthma,asthma,asthma_Zip5_6,avg_enc_asthma,0.0,9xs2-f89t,Chicago Public Schools - Progress Report Cards...,9xs2-f89t_location_6,avg_general_services_route_,0.00,0.555,0.000,46,spatial
5,asthma,asthma,asthma_Zip5_6,avg_encAsthmaExac,0.0,9xs2-f89t,Chicago Public Schools - Progress Report Cards...,9xs2-f89t_location_6,avg_general_services_route_,0.00,0.509,0.000,46,spatial
6,asthma,asthma,asthma_Zip5_6,avg_enc_asthma,0.0,4aki-r3np,Strategic Subject List - Historical,4aki-r3np_location_6,avg_predictor_rat_arrests_violent_offenses,0.00,0.719,0.000,49,spatial
7,asthma,asthma,asthma_Zip5_6,avg_enc_asthma,0.0,4aki-r3np,Strategic Subject List - Historical,4aki-r3np_location_6,avg_predictor_rat_uuw_arrests,0.00,0.702,0.000,49,spatial
8,asthma,asthma,asthma_Zip5_6,avg_encAsthmaExac,0.0,4aki-r3np,Strategic Subject List - Historical,4aki-r3np_location_6,avg_predictor_rat_arrests_violent_offenses,0.00,0.696,0.000,49,spatial
9,asthma,asthma,asthma_Zip5_6,avg_encAsthmaExac,0.0,4aki-r3np,Strategic Subject List - Historical,4aki-r3np_location_6,avg_predictor_rat_uuw_arrests,0.00,0.667,0.000,49,spatial


Button(description='Download Data', style=ButtonStyle())

# Assemble a dataset from multiple variables

1. df, prov = get_aligned_data(correlation): this api takes input as a correlation and returns the merged dataset used to calculate this correlation

2. df, prov = assemble(vars, constraints: [optional]): this api creates a dataset that merges all variables specified in `vars`. `constaints` is a mapping between table name and the constraint on that table when performing join operation. For example {'tbl_A': 2} means spatio-temporal units with the number of samples smaller than 2 are discarded. 

data assembly APIs return `prov`, which is the provenance information of the resulting dataset.


In [10]:
row_idx = 0
aligned, prov = nexus_api.get_joined_data_from_row(correlations.loc[0])
show_df(aligned, name="asthma_corrs_aligned", prov=prov, use_qgrid=use_qgrid)

,val,avg_enc_asthma,count
0,60626,36.0,14570
1,60605,47.0,11297
2,60640,15.0,16492
3,60613,12.0,9932
4,60643,964.0,17343
5,60621,1288.0,27198
6,60616,269.0,18038
7,60608,83.0,21219
8,60647,36.0,20587
9,60625,16.0,14754


provenance:


'\n                SELECT a1.val, a1."avg_enc_asthma" AS "avg_enc_asthma",a2."count" AS "count" FROM\n                "asthma_Zip5_6" a1 JOIN "ijzp-q8t2_location_6" a2\n                ON a1.val = a2.val\n            '

Button(description='Download Data', style=ButtonStyle())

In [11]:
# without constraint
vars = [Variable('divg-mhqk_location_6', 'count'), Variable('4u6w-irs9_location_6', 'avg_square_feet')]
df, prov = nexus_api.join_and_project(vars)
show_df(df, name="divg-mhqk_4u6w-irs9", prov=prov, use_qgrid=use_qgrid)

,count,avg_square_feet,divg-mhqk_location_6_samples,4u6w-irs9_location_6_samples
0,1,12457.583333,1,12
1,1,6833.125000,1,8
2,1,15612.687500,1,16
3,1,14105.125000,1,8
4,1,14639.500000,1,6
5,5,6096.285714,5,21
6,2,8869.909091,2,11
7,1,11391.888889,1,18
8,1,17923.250000,1,4


provenance:


'SELECT "divg-mhqk_location_6".count,"4u6w-irs9_location_6".avg_square_feet,"divg-mhqk_location_6".count AS "divg-mhqk_location_6_samples","4u6w-irs9_location_6".count AS "4u6w-irs9_location_6_samples" FROM "divg-mhqk_location_6" INNER JOIN "4u6w-irs9_location_6" ON "divg-mhqk_location_6".val = "4u6w-irs9_location_6".val'

Button(description='Download Data', style=ButtonStyle())

In [12]:
# with constraint, units with number of samples smaller than 2 are dropped
vars = [Variable('divg-mhqk_location_6', 'count'), Variable('4u6w-irs9_location_6', 'avg_square_feet')]
constraints = {'divg-mhqk_location_6': 2, '4u6w-irs9_location_6': 2}
df, prov = nexus_api.join_and_project(vars, constraints)
show_df(df, name="divg-mhqk_4u6w-irs9_sample_greater_than_2", prov=prov, use_qgrid=use_qgrid)

,count,avg_square_feet,divg-mhqk_location_6_samples,4u6w-irs9_location_6_samples
0,5,6096.285714,5,21
1,2,8869.909091,2,11


provenance:


'SELECT "divg-mhqk_location_6".count,"4u6w-irs9_location_6".avg_square_feet,"divg-mhqk_location_6".count AS "divg-mhqk_location_6_samples","4u6w-irs9_location_6".count AS "4u6w-irs9_location_6_samples" FROM "divg-mhqk_location_6" INNER JOIN "4u6w-irs9_location_6" ON "divg-mhqk_location_6".val = "4u6w-irs9_location_6".val WHERE "divg-mhqk_location_6".count >= 2 AND "4u6w-irs9_location_6".count >= 2'

Button(description='Download Data', style=ButtonStyle())

# Regression Analysis

When you find multiple intriguing correlations and wish to conduct further regression analysis on variables of interest, you can begin by utilizing Nexus's `join_and_project` function to compile the necessary dataset. Subsequently, you may employ any data analysis library for regression analysis. In this instance, we will illustrate the process using `scikit-learn`.

In [13]:
dependent_variable = Variable('asthma_Zip5_6', 'avg_enc_asthma')
independent_variables = [Variable('ijzp-q8t2_location_6', 'count'), Variable('n26f-ihde_pickup_centroid_location_6', 'avg_tip')]

data_to_analyze, provenance = nexus_api.join_and_project([dependent_variable] + independent_variables)
# apply any data anlysis method
regression_model = linear_model.LinearRegression() # OLS regression

x = data_to_analyze[[variable.attr_name for variable in independent_variables]]
y = data_to_analyze[dependent_variable.attr_name]
model = regression_model.fit(x, y)
r_squared = model.score(x, y)

print("coefficients of each independent variables:", model.coef_)
print("r square score:", r_squared)

coefficients of each independent variables: [ 3.17139472e-02 -5.13593106e+02]
r square score: 0.34087326231772097


# Analyze Correlations

In this section, we will explain how to analyze output correlations in Nexus. We will use the correlations from chicago open data at the census tract and month granularity as an example.

In [14]:
# load correlations: corrs is a list of correlations; corr_map is map from correlated variables to their correlation coefficients
corr_path = "evaluation/correlations2/chicago_1m_T_GRANU.MONTH_S_GRANU.TRACT/"
corrs, corr_map = load_corrs_from_dir(corr_path) 

## Use Nexus Variable Clusters

Nexus searches for an optimal set of signals that, when applied as filters, yield a correlation graph with the highest modularity score. The signals that we consider for chicago open data include:

- Missing value ratio in the aggregated column
- Missing value ratio in the original column
- Zero value ratio in the aggregated column
- Zero value ratio in the original column
- The absolute value of correlation coefficient
- Overlap: number of samples used to calculate the correlation

In chicago open data, the best set of thresholds for the above signals are [1.0, 1.0, 1.0, 0.8, 0.6, 70], which means we include correlations whose missing_ratio <= 1.0, missing_ratio_original<=1.0, zero_ratio <=1.0, zero_ratio_original <= 0.8, |r| >= 0.6, |samples| >= 70.`

You can play with different set of thresholds as well!

In [15]:
signal_thresholds = [1.0, 1.0, 1.0, 0.8, 0.6, 70]
corr_community = CorrCommunity(corrs, 'chicago')
corr_community.get_correlation_communities_chicago(signal_thresholds)

covered #tbls: 163, original #tbls: 264, coverage ratio: 0.6174242424242424, modularity score: 0.6724097704513528
covered #correlations: 12439, original #correlations: 46143


### Examine Correlation Communities

We implement a simple interface for you to explore our correlation communities. Each community is composed of a group of variables. By default, the display is set to only show the tables where these variables are found. To view the specific variables within a community, simply click the "Show Variables" button.

Clicking the "Show Correlations" button will reveal all the correlations within a community. Once displayed, you have the flexibility to apply any filters to the resulting dataframe.

FAQ:

Why do some communities display the exact same set of tables?

The reason is that while the tables might be the same, the variables within these communities differ. We construct the correlation graph based on variables, and then present it in a table-view for clarity.

In [16]:
show_communities(corr_community, show_corr_in_same_tbl=False, use_qgrid=use_qgrid)

Dropdown(description='Show:', layout=Layout(width='200px'), options=('Cluster 0', 'Cluster 1', 'Cluster 2', 'C…

Output()

Output()

Output()

## Use Factor Analysis

Factor analysis aims to extract common factors from observed variables and represent existing variables using fewer factors. 

Factor analysis can take as input a correlation matrix. It derives factors that are essentially linear combinations of the observed variables. These factors are crafted to closely approximate the original correlation matrix when observed variables are projected onto them. 

In [18]:
# need to remove correlations that have values of 1 or -1 to avoid singular matrix
corrs, corr_map = load_corrs_from_dir(corr_path, remove_perfect_corrs=True) 
signals = [1.0, 1.0, 1.0, 0.8, 0.6, 70] # we use the same signal thresholds as in the previous example
corrs_filtered = filter_on_signals(corrs, None, signals)

n_factors = 10 # set the number of factors to 10

"""
the following code fits a factor analysis model on the correlation matrix online
It takes 10 minutes to run; save_path indicates the path to save the factor analysis model (fa)
"""
#fa, clusters = nexus_api.factor_analysis(corrs_filtered, corr_map, n_factors, save_path="chicago_open_data_factor_analysis.pkl")

"""
For the purpose of this demo, we load the factor analysis model from the file "chicago_open_data_factor_analysis.pkl".
You may need to fit a factor analysis model yourself in the above line of code if you are running this for the first time.
"""
fa = pickle.load(open("chicago_open_data_factor_analysis.pkl", "rb"))
clusters, covered_vars = nexus_api.build_factor_clusters(fa, corrs_filtered, corr_map, n_factors, threshold=0.5)
corr_community = CorrCommunity(corrs_filtered, 'chicago', clusters)
show_communities(corr_community, show_corr_in_same_tbl=False, use_qgrid=use_qgrid)

Dropdown(description='Show:', layout=Layout(width='200px'), options=('Cluster 0', 'Cluster 1', 'Cluster 2', 'C…

Output()

Output()

Output()